In [12]:
import sklearn as sk
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [92]:
# Loading data and check if it's loaded properly
df = pd.read_csv("Paitients_Files_Train.csv", delimiter=",")
print(df.head(5))

          ID  PRG   PL  PR  SK   TS   M11    BD2  Age  Insurance   Sepssis
0  ICU200010    6  148  72  35    0  33.6  0.627   50          0  Positive
1  ICU200011    1   85  66  29    0  26.6  0.351   31          0  Negative
2  ICU200012    8  183  64   0    0  23.3  0.672   32          1  Positive
3  ICU200013    1   89  66  23   94  28.1  0.167   21          1  Negative
4  ICU200014    0  137  40  35  168  43.1  2.288   33          1  Positive


In [14]:
# drop 2 irrelevant columns: ID and Insurance
df.drop(columns="ID", inplace=True)
df.drop(columns="Insurance", inplace=True)

In [15]:
X = df.drop(columns="Sepssis")
Y = df[["Sepssis"]]

In [90]:
Y.value_counts()

Sepssis 
Negative    391
Positive    208
dtype: int64

In [16]:
# spliting
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=6)
print("Training X:", X_train.shape)
print("Testing X: ",X_test.shape)
print("Train Y: ",y_train.shape)

Training X: (449, 8)
Testing X:  (150, 8)
Train Y:  (449, 1)


In [85]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn import metrics
dtf = DecisionTreeClassifier(criterion="gini",max_features=2/3, max_depth=5, random_state=3, class_weight={"Negative":1, "Positive": 2})
# fit model

dtf.fit(X_train, y_train)
# make predictions

yhat = dtf.predict(X_test)

# calculate accuracy

accuracy = metrics.accuracy_score(yhat, y_test)

print('Accuracy: %.3f' % dtf.score(X_test, y_test))

Accuracy: 0.767


In [86]:
text_ = tree.export_text(dtf,feature_names=['BGlucose', 'B1', 'BPressure', 'B2', 'B3', 'BMI', 'B4', 'Age'])
print(text_)

|--- BMI <= 30.25
|   |--- B1 <= 103.50
|   |   |--- BGlucose <= 11.50
|   |   |   |--- BMI <= 28.95
|   |   |   |   |--- class: Negative
|   |   |   |--- BMI >  28.95
|   |   |   |   |--- BMI <= 29.10
|   |   |   |   |   |--- class: Negative
|   |   |   |   |--- BMI >  29.10
|   |   |   |   |   |--- class: Negative
|   |   |--- BGlucose >  11.50
|   |   |   |--- class: Positive
|   |--- B1 >  103.50
|   |   |--- BMI <= 25.45
|   |   |   |--- B1 <= 177.50
|   |   |   |   |--- BMI <= 9.20
|   |   |   |   |   |--- class: Positive
|   |   |   |   |--- BMI >  9.20
|   |   |   |   |   |--- class: Negative
|   |   |   |--- B1 >  177.50
|   |   |   |   |--- class: Positive
|   |   |--- BMI >  25.45
|   |   |   |--- Age <= 62.50
|   |   |   |   |--- B4 <= 0.25
|   |   |   |   |   |--- class: Negative
|   |   |   |   |--- B4 >  0.25
|   |   |   |   |   |--- class: Positive
|   |   |   |--- Age >  62.50
|   |   |   |   |--- class: Negative
|--- BMI >  30.25
|   |--- B1 <= 127.50
|   |   |--- Age

In [87]:
f_probs = dtf.predict_proba(X_test)
f_probs = f_probs[:,1]
print(metrics.roc_auc_score(y_test, f_probs))

0.8445236680530799


In [88]:
print(metrics.classification_report(y_test, yhat))

              precision    recall  f1-score   support

    Negative       0.90      0.73      0.80        99
    Positive       0.61      0.84      0.71        51

    accuracy                           0.77       150
   macro avg       0.76      0.79      0.76       150
weighted avg       0.80      0.77      0.77       150

